# Scraping

## Importation des packages

In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common import TimeoutException
from bs4 import BeautifulSoup
import requests

C:\Users\Nicolas et Maëva\.conda\envs\Workspace\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Nicolas et Maëva\.conda\envs\Workspace\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\Nicolas et Maëva\.conda\envs\Workspace\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Scraping des données pour la première page web

In [2]:
url = 'https://ceoworld.biz/2023/08/28/ranked-countries-with-the-highest-share-of-women-in-parliament/'

Grâce à l'inspecteur de notre navigateur, nous voyons que tout le tableau est contenu dans la balise :
`<table id="tablepress-674" class="tablepress tablepress-id-674 tablepress-responsive dataTable no-footer" aria-labelledby="tablepress-674-name" aria-describedby="tablepress-674_info" style="width: 670px;">`,
et que chaque ligne est contenue dans une balise : `<tr class="even">` ou `<tr class="odd">`.
Finalement, chaque valeur du tableau est contenue dans une balise `<td>`.

In [3]:
soup = BeautifulSoup(requests.get(url).content)
table_soup = soup.find_all('table', recursive=True)
table_soup

[<table aria-labelledby="tablepress-674-name" class="tablepress tablepress-id-674 tablepress-responsive" id="tablepress-674">
 <thead>
 <tr class="row-1 odd">
 <th class="column-1">Global rank </th><th class="column-2">Countries </th><th class="column-3">Women in parliament</th>
 </tr>
 </thead>
 <tbody class="row-hover">
 <tr class="row-2 even">
 <td class="column-1">1</td><td class="column-2">Rwanda</td><td class="column-3">61.25</td>
 </tr>
 <tr class="row-3 odd">
 <td class="column-1">2</td><td class="column-2">Nicaragua</td><td class="column-3">51.65</td>
 </tr>
 <tr class="row-4 even">
 <td class="column-1">3</td><td class="column-2">New Zealand</td><td class="column-3">50.42</td>
 </tr>
 <tr class="row-5 odd">
 <td class="column-1">4</td><td class="column-2">Mexico</td><td class="column-3">50</td>
 </tr>
 <tr class="row-6 even">
 <td class="column-1">5</td><td class="column-2">United Arab Emirates</td><td class="column-3">50</td>
 </tr>
 <tr class="row-7 odd">
 <td class="column

Malheureusement, nous n'arrivons pas à extraire toutes les lignes du tableau, car la page web exécute du code JavaScript, et de plus, nous devons appuyer sur le bouton `next` pour accéder aux pages suivantes. Nous devrons donc utiliser Selenium, qui réplique le comportement d'un utilisateur, afin d'extraire les données.

In [4]:
driver = webdriver.Chrome()
driver.get(url)
data = []
time.sleep(10)
while True:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    data.append(soup.find_all('table'))

    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#tablepress-674_next"))).click() 
        # Le driver trouve le bouton grâce au sélecteur CSS, et attend 10 secondes maximum que celui-ci devienne cliquable. Au bout des dix secondes, si le bouton n'est toujours pas cliquable le code passe au bloc except TimeoutException.

        time.sleep(2) # Le driver attend, afin d'être sûr que la nouvelle page soit chargée.
        
    except TimeoutException:
        break  # Si ce code s'exécute, c'est que le driver n'a pas pu cliquer sur le bouton, il n'y a donc plus de pages, donc nous arrêtons la boucle.

    finally:
        driver.quit()

MaxRetryError: HTTPConnectionPool(host='localhost', port=63397): Max retries exceeded with url: /session/8ee3cc4b43f0e06c0613444efe0e45c5/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020F7D1B5F30>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))

Maintenant, filtrons le résultat brut que nous avons obtenu.

In [5]:
data_first_filtering=[]
for result_set in data:
    for tag in result_set:
        rows = tag.find_all('tr', {'class' : ['even', 'odd']})
        # On récupère les lignes correspondant à notre filtre
        for row in rows:
            if row.find('th') is None:
                data_first_filtering.append(row)
        # On n'append que si la ligne ne contient pas de balise th
data_first_filtering

[<tr class="even"><td>1</td><td>Rwanda</td><td>61.25</td></tr>,
 <tr class="odd"><td>2</td><td>Nicaragua</td><td>51.65</td></tr>,
 <tr class="even"><td>3</td><td>New Zealand</td><td>50.42</td></tr>,
 <tr class="odd"><td>4</td><td>Mexico</td><td>50</td></tr>,
 <tr class="even"><td>5</td><td>United Arab Emirates</td><td>50</td></tr>,
 <tr class="odd"><td>6</td><td>Iceland</td><td>47.62</td></tr>,
 <tr class="even"><td>7</td><td>Costa Rica</td><td>47.37</td></tr>,
 <tr class="odd"><td>8</td><td>South Africa</td><td>46.5</td></tr>,
 <tr class="even"><td>9</td><td>Andorra</td><td>46.43</td></tr>,
 <tr class="odd"><td>10</td><td>Sweden</td><td>46.42</td></tr>,
 <tr class="even"><td>11</td><td>Bolivia</td><td>46.15</td></tr>,
 <tr class="odd"><td>12</td><td>Finland</td><td>45.5</td></tr>,
 <tr class="even"><td>13</td><td>Norway</td><td>44.97</td></tr>,
 <tr class="odd"><td>14</td><td>Argentina</td><td>44.75</td></tr>,
 <tr class="even"><td>15</td><td>Senegal</td><td>44.24</td></tr>,
 <tr clas

Maintenant, nous devons extraire chaque élément contenu dans les balises td pour chaque balise tr.

In [6]:
data_second_filtering = list([x.find_all('td') for x in data_first_filtering])
data_second_filtering

[[<td>1</td>, <td>Rwanda</td>, <td>61.25</td>],
 [<td>2</td>, <td>Nicaragua</td>, <td>51.65</td>],
 [<td>3</td>, <td>New Zealand</td>, <td>50.42</td>],
 [<td>4</td>, <td>Mexico</td>, <td>50</td>],
 [<td>5</td>, <td>United Arab Emirates</td>, <td>50</td>],
 [<td>6</td>, <td>Iceland</td>, <td>47.62</td>],
 [<td>7</td>, <td>Costa Rica</td>, <td>47.37</td>],
 [<td>8</td>, <td>South Africa</td>, <td>46.5</td>],
 [<td>9</td>, <td>Andorra</td>, <td>46.43</td>],
 [<td>10</td>, <td>Sweden</td>, <td>46.42</td>],
 [<td>11</td>, <td>Bolivia</td>, <td>46.15</td>],
 [<td>12</td>, <td>Finland</td>, <td>45.5</td>],
 [<td>13</td>, <td>Norway</td>, <td>44.97</td>],
 [<td>14</td>, <td>Argentina</td>, <td>44.75</td>],
 [<td>15</td>, <td>Senegal</td>, <td>44.24</td>],
 [<td>16</td>, <td>Namibia</td>, <td>44.23</td>],
 [<td>17</td>, <td>Denmark</td>, <td>43.58</td>],
 [<td>18</td>, <td>Spain</td>, <td>42.98</td>],
 [<td>19</td>, <td>Belgium</td>, <td>42.67</td>],
 [<td>20</td>, <td>Mozambique</td>, <td>42.4

Rangeons les valeurs des trois colonnes dans trois listes distinctes

In [7]:
(
global_rank,
country,
women_parliament_percentage
)\
=\
(
[str(x[0]).replace('<td>', '').replace('</td>', '') for x in data_second_filtering],
[str(x[1]).replace('<td>', '').replace('</td>', '') for x in data_second_filtering],
[str(x[2]).replace('<td>', '').replace('</td>', '') for x in data_second_filtering]
)

Nous avons maintenant trois listes que nous allons entrer dans un DataFrame

In [8]:
# Transformation de nos listes en séries pour permettre la concaténation
global_rank = pd.Series(global_rank)
country = pd.Series(country)
women_parliament_percentage = pd.Series(women_parliament_percentage)

#Concaténation
df_parliament = pd.concat([global_rank, country, women_parliament_percentage], axis=1)
df_parliament.columns = ['Global Rank', 'Country', 'Women Parliament Percentage']
df_parliament

,Global Rank,Country,Women Parliament Percentage
0,1,Rwanda,61.25
1,2,Nicaragua,51.65
2,3,New Zealand,50.42
3,4,Mexico,50
4,5,United Arab Emirates,50
5,6,Iceland,47.62
6,7,Costa Rica,47.37
7,8,South Africa,46.5
8,9,Andorra,46.43
9,10,Sweden,46.42


Notre premier dataframe est prêt, nous allons maintenant répéter l'opération pour une deuxième page.

## Scraping des données pour la deuxième page web

In [9]:
url = 'https://ceoworld.biz/2021/06/11/the-worlds-best-countries-for-women-2021/'

In [10]:
df_score = pd.read_html(url)[0]
df_score

,Rank,Country,Score
0,1,Netherlands,99.70
1,2,Norway,99.40
2,3,Sweden,99.20
3,4,Denmark,98.70
4,5,Finland,98.30
...,...,...,...
151,152,Niger,23.56
152,153,Syria,21.98
153,154,Chad,19.30
154,155,Somalia,14.68


In [11]:
df_analyse = pd.merge(df_parliament, df_score, how='inner', on='Country')
df_analyse.drop(columns=['Global Rank', 'Rank'], inplace=True)
df_analyse

,Country,Women Parliament Percentage,Score
0,New Zealand,50.42,96.80
1,Mexico,50,83.79
2,United Arab Emirates,50,73.16
3,Iceland,47.62,88.73
4,Costa Rica,47.37,77.75
5,South Africa,46.5,71.01
6,Andorra,46.43,86.60
7,Sweden,46.42,99.20
8,Bolivia,46.15,77.95
9,Finland,45.5,98.30


Maintenant exportons le DataFrame en fichier CSV afin de l'importer ensuite dans PowerBI

In [12]:
df_analyse.to_csv('df_analyse.csv', index=False)
df_analyse.to_csv('df_analyse.csv', index=False)